<a href="https://colab.research.google.com/github/AI4Finance-Foundation/FinRL-Tutorials/blob/master/1-Introduction/Stock_NeurIPS2018_SB3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Reinforcement Learning for Stock Trading from Scratch: Multiple Stock Trading

* **Pytorch Version** 



# Content

* [1. Task Description](#0)
* [2. Install Python packages](#1)
    * [2.1. Install Packages](#1.1)    
    * [2.2. A List of Python Packages](#1.2)
    * [2.3. Import Packages](#1.3)
    * [2.4. Create Folders](#1.4)
* [3. Download and Preprocess Data](#2)
* [4. Preprocess Data](#3)        
    * [4.1. Technical Indicators](#3.1)
    * [4.2. Perform Feature Engineering](#3.2)
* [5. Build Market Environment in OpenAI Gym-style](#4)  
    * [5.1. Data Split](#4.1)  
    * [5.3. Environment for Training](#4.2)    
* [6. Train DRL Agents](#5)
* [7. Backtesting Performance](#6)  
    * [7.1. BackTestStats](#6.1)
    * [7.2. BackTestPlot](#6.2)   
  

<a id='0'></a>
# Part 1. Task Discription

We train a DRL agent for stock trading. This task is modeled as a Markov Decision Process (MDP), and the objective function is maximizing (expected) cumulative return.

We specify the state-action-reward as follows:

* **State s**: The state space represents an agent's perception of the market environment. Just like a human trader analyzing various information, here our agent passively observes many features and learns by interacting with the market environment (usually by replaying historical data).

* **Action a**: The action space includes allowed actions that an agent can take at each state. For example, a ∈ {−1, 0, 1}, where −1, 0, 1 represent
selling, holding, and buying. When an action operates multiple shares, a ∈{−k, ..., −1, 0, 1, ..., k}, e.g.. "Buy
10 shares of AAPL" or "Sell 10 shares of AAPL" are 10 or −10, respectively

* **Reward function r(s, a, s′)**: Reward is an incentive for an agent to learn a better policy. For example, it can be the change of the portfolio value when taking a at state s and arriving at new state s',  i.e., r(s, a, s′) = v′ − v, where v′ and v represent the portfolio values at state s′ and s, respectively


**Market environment**: 30 consituent stocks of Dow Jones Industrial Average (DJIA) index. Accessed at the starting date of the testing period.


The data for this case study is obtained from Yahoo Finance API. The data contains Open-High-Low-Close price and volume.


<a id='1'></a>
# Part 2. Install Python Packages

<a id='1.1'></a>
## 2.1. Install packages


In [1]:
## install required packages
!pip install swig
!pip install wrds
!pip install pyportfolioopt
## install finrl library
!pip install -q condacolab
import condacolab
condacolab.install()
!apt-get update -y -qq && apt-get install -y -qq cmake libopenmpi-dev python3-dev zlib1g-dev libgl1-mesa-glx swig
!pip install git+https://github.com/AI4Finance-Foundation/FinRL.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached swig-4.1.1-py2.py3-none-manylinux_2_5_x86_64.manylinux1_x86_64.whl (1.8 MB)
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached pyportfolioopt-1.5.4-py3-none-any.whl (61 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 29.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.1/220.1 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 41.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.2/298.2 kB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 45.2 MB/s eta 0:00:00
✨🍰✨ Everything looks OK!
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.c


<a id='1.2'></a>
## 2.2. A list of Python packages 
* Yahoo Finance API
* pandas
* numpy
* matplotlib
* stockstats
* OpenAI gym
* stable-baselines
* tensorflow
* pyfolio

<a id='1.3'></a>
## 2.3. Import Packages

In [2]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
# matplotlib.use('Agg')
import datetime

%matplotlib inline
from finrl.meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.meta.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl.meta.env_stock_trading.env_stocktrading import StockTradingEnv
from finrl.agents.stablebaselines3.models import DRLAgent
from stable_baselines3.common.logger import configure
from finrl.meta.data_processor import DataProcessor

from finrl.plot import backtest_stats, backtest_plot, get_daily_return, get_baseline
from pprint import pprint

import sys
sys.path.append("../FinRL")

import itertools

/usr/local/lib/python3.9/site-packages/pyfolio/pos.py:26: UserWarning: Module "zipline.assets" not found; multipliers will not be applied to position notionals.
  warnings.warn(


<a id='1.4'></a>
## 2.4. Create Folders

In [3]:
from finrl import config
from finrl import config_tickers
import os
from finrl.main import check_and_make_directories
from finrl.config import (
    DATA_SAVE_DIR,
    TRAINED_MODEL_DIR,
    TENSORBOARD_LOG_DIR,
    RESULTS_DIR,
    INDICATORS,
    TRAIN_START_DATE,
    TRAIN_END_DATE,
    TEST_START_DATE,
    TEST_END_DATE,
    TRADE_START_DATE,
    TRADE_END_DATE,
)
check_and_make_directories([DATA_SAVE_DIR, TRAINED_MODEL_DIR, TENSORBOARD_LOG_DIR, RESULTS_DIR])



<a id='2'></a>
# Part 3. Download Data
Yahoo Finance provides stock data, financial news, financial reports, etc. Yahoo Finance is free.
* FinRL uses a class **YahooDownloader** in FinRL-Meta to fetch data via Yahoo Finance API
* Call Limit: Using the Public API (without authentication), you are limited to 2,000 requests per hour per IP (or up to a total of 48,000 requests a day).



-----
class YahooDownloader:
    Retrieving daily stock data from
    Yahoo Finance API

    Attributes
    ----------
        start_date : str
            start date of the data (modified from config.py)
        end_date : str
            end date of the data (modified from config.py)
        ticker_list : list
            a list of stock tickers (modified from config.py)

    Methods
    -------
    fetch_data()


In [87]:
# from config.py, TRAIN_START_DATE is a string
TRAIN_START_DATE
# from config.py, TRAIN_END_DATE is a string
TRAIN_END_DATE

'2021-10-01'

In [88]:
TRAIN_START_DATE = '2010-01-01'
TRAIN_END_DATE = '2021-10-01'
TRADE_START_DATE = '2021-10-01'
TRADE_END_DATE = '2023-03-01'

In [89]:
df = YahooDownloader(start_date = TRAIN_START_DATE,
                     end_date = TRADE_END_DATE,
                     ticker_list = config_tickers.DOW_30_TICKER).fetch_data()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [90]:
print(config_tickers.DOW_30_TICKER)

['AXP', 'AMGN', 'AAPL', 'BA', 'CAT', 'CSCO', 'CVX', 'GS', 'HD', 'HON', 'IBM', 'INTC', 'JNJ', 'KO', 'JPM', 'MCD', 'MMM', 'MRK', 'MSFT', 'NKE', 'PG', 'TRV', 'UNH', 'CRM', 'VZ', 'V', 'WBA', 'WMT', 'DIS', 'DOW']


In [91]:
df.shape

(97013, 8)

In [92]:
df.sort_values(['date','tic'],ignore_index=True).head()

date       open       high        low      close     volume   tic   
0  2010-01-04   7.622500   7.660714   7.585000   6.505281  493729600  AAPL  \
1  2010-01-04  56.630001  57.869999  56.560001  42.888958    5277400  AMGN   
2  2010-01-04  40.810001  41.099998  40.389999  33.551674    6894300   AXP   
3  2010-01-04  55.720001  56.389999  54.799999  43.777550    6186700    BA   
4  2010-01-04  57.650002  59.189999  57.509998  41.156906    7325600   CAT   

   day  
0    0  
1    0  
2    0  
3    0  
4    0

# Part 4: Preprocess Data
We need to check for missing data and do feature engineering to convert the data point into a state.
* **Adding technical indicators**. In practical trading, various information needs to be taken into account, such as historical prices, current holding shares, technical indicators, etc. Here, we demonstrate two trend-following technical indicators: MACD and RSI.
* **Adding turbulence index**. Risk-aversion reflects whether an investor prefers to protect the capital. It also influences one's trading strategy when facing different market volatility level. To control the risk in a worst-case scenario, such as financial crisis of 2007–2008, FinRL employs the turbulence index that measures extreme fluctuation of asset price.

In [93]:
fe = FeatureEngineer(
                    use_technical_indicator=True,
                    tech_indicator_list = INDICATORS,
                    use_vix=True,
                    use_turbulence=True,
                    user_defined_feature = False)

processed = fe.preprocess_data(df)

Successfully added technical indicators
[*********************100%***********************]  1 of 1 completed
Shape of DataFrame:  (3310, 8)
Successfully added vix
Successfully added turbulence index


In [94]:
list_ticker = processed["tic"].unique().tolist()
list_date = list(pd.date_range(processed['date'].min(),processed['date'].max()).astype(str))
combination = list(itertools.product(list_date,list_ticker))

processed_full = pd.DataFrame(combination,columns=["date","tic"]).merge(processed,on=["date","tic"],how="left")
processed_full = processed_full[processed_full['date'].isin(processed['date'])]
processed_full = processed_full.sort_values(['date','tic'])

processed_full = processed_full.fillna(0)

In [95]:
processed_full.sort_values(['date','tic'],ignore_index=True).head(10)

date   tic        open        high         low       close   
0  2010-01-04  AAPL    7.622500    7.660714    7.585000    6.505281  \
1  2010-01-04  AMGN   56.630001   57.869999   56.560001   42.888958   
2  2010-01-04   AXP   40.810001   41.099998   40.389999   33.551674   
3  2010-01-04    BA   55.720001   56.389999   54.799999   43.777550   
4  2010-01-04   CAT   57.650002   59.189999   57.509998   41.156906   
5  2010-01-04   CRM   18.652500   18.882500   18.547501   18.705000   
6  2010-01-04  CSCO   24.110001   24.840000   24.010000   17.264450   
7  2010-01-04   CVX   78.199997   79.199997   78.160004   46.851871   
8  2010-01-04   DIS   32.500000   32.750000   31.870001   27.933922   
9  2010-01-04    GS  170.050003  174.250000  169.509995  139.862900   

        volume  day  macd   boll_ub  boll_lb  rsi_30     cci_30  dx_30   
0  493729600.0  0.0   0.0  6.526808    6.495   100.0  66.666667  100.0  \
1    5277400.0  0.0   0.0  6.526808    6.495   100.0  66.666667  100.0   
2    6894300.0  0.0   0.0  6.526808    6.495   100.0  66.666667  100.0   
3    6186700.0  0.0   0.0  6.526808    6.495   100.0  66.666667  100.0   
4    7325600.0  0.0   0.0  6.526808    6.495   100.0  66.666667  100.0   
5    7906000.0  0.0   0.0  6.526808    6.495   100.0  66.666667  100.0   
6   59853700.0  0.0   0.0  6.526808    6.495   100.0  66.666667  100.0   
7   10173800.0  0.0   0.0  6.526808    6.495   100.0  66.666667  100.0   
8   13700400.0  0.0   0.0  6.526808    6.495   100.0  66.666667  100.0   
9    9135000.0  0.0   0.0  6.526808    6.495   100.0  66.666667  100.0   

   close_30_sma  close_60_sma        vix  turbulence  
0      6.505281      6.505281  20.040001         0.0  
1     42.888958     42.888958  20.040001         0.0  
2     33.551674     33.551674  20.040001         0.0  
3     43.777550     43.777550  20.040001         0.0  
4     41.156906     41.156906  20.040001         0.0  
5     18.705000     18.705000  20.040001         0.0  
6     17.264450     17.264450  20.040001         0.0  
7     46.851871     46.851871  20.040001         0.0  
8     27.933922     27.933922  20.040001         0.0  
9    139.862900    139.862900  20.040001         0.0

In [96]:
mvo_df = processed_full.sort_values(['date','tic'],ignore_index=True)[['date','tic','close']]

<a id='4'></a>
# Part 5. Build A Market Environment in OpenAI Gym-style
The training process involves observing stock price change, taking an action and reward's calculation. By interacting with the market environment, the agent will eventually derive a trading strategy that may maximize (expected) rewards.

Our market environment, based on OpenAI Gym, simulates stock markets with historical market data.

## Data Split
We split the data into training set and testing set as follows:

Training data period: 2009-01-01 to 2020-07-01

Trading data period: 2020-07-01 to 2021-10-31


In [97]:
train = data_split(processed_full, TRAIN_START_DATE,TRAIN_END_DATE)
trade = data_split(processed_full, TRADE_START_DATE,TRADE_END_DATE)
train_length = len(train)
trade_length = len(trade)
print(train_length)
print(trade_length)

85753
10237


In [98]:
train.tail()

date  tic        open        high         low       close   
2956  2021-09-30  UNH  401.489990  403.489990  390.459991  383.180176  \
2956  2021-09-30    V  227.580002  228.789993  222.630005  220.257767   
2956  2021-09-30   VZ   54.500000   54.509998   54.000000   49.011551   
2956  2021-09-30  WBA   48.790001   48.930000   46.919998   43.957283   
2956  2021-09-30  WMT  140.639999  141.729996  139.250000  136.114777   

          volume  day      macd     boll_ub     boll_lb     rsi_30   
2956   3779900.0  3.0 -4.349256  419.212270  386.863774  40.895395  \
2956   7128500.0  3.0 -1.538728  228.639270  216.529608  44.078996   
2956  18736600.0  3.0 -0.233667   50.131163   48.744038  41.824917   
2956   6449400.0  3.0 -0.253540   48.531810   43.597246  44.613711   
2956   7485900.0  3.0 -1.554834  146.155111  135.633136  40.165878   

          cci_30      dx_30  close_30_sma  close_60_sma        vix  turbulence  
2956 -222.938238  41.980385    405.947466    405.438423  23.139999   24.872328  
2956  -54.614579  19.569853    224.777295    231.479431  23.139999   24.872328  
2956 -102.798842  21.682953     49.605026     50.130592  23.139999   24.872328  
2956 -107.390223   0.941150     45.886550     44.857786  23.139999   24.872328  
2956 -151.542656  45.466733    142.345313    141.246727  23.139999   24.872328

In [99]:
trade.head()

date   tic        open        high         low       close   
0  2021-10-01  AAPL  141.899994  142.919998  139.110001  141.404266  \
0  2021-10-01  AMGN  213.589996  214.610001  210.800003  203.845871   
0  2021-10-01   AXP  168.500000  175.119995  168.479996  170.065353   
0  2021-10-01    BA  222.850006  226.720001  220.600006  226.000000   
0  2021-10-01   CAT  192.899994  195.869995  191.240005  187.928040   

       volume  day      macd     boll_ub     boll_lb     rsi_30      cci_30   
0  94639600.0  4.0 -1.703488  155.382846  137.132193  46.927735 -142.190202  \
0   2629400.0  4.0 -3.097330  212.767980  199.379595  40.408533  -96.757039   
0   3956000.0  4.0  2.273329  174.218856  149.232889  56.265093  117.538402   
0   9113600.0  4.0  0.730320  226.909442  205.727561  51.614047  116.649440   
0   3695500.0  4.0 -3.640324  205.735919  181.432783  41.999435 -112.087765   

       dx_30  close_30_sma  close_60_sma   vix  turbulence  
0  41.749873    147.171798    146.269415  21.1  120.122978  
0  36.189244    208.480832    217.103342  21.1  120.122978  
0  15.667511    161.215661    163.458888  21.1  120.122978  
0   2.027170    217.175334    221.968500  21.1  120.122978  
0  36.203176    196.993869    200.522109  21.1  120.122978

In [100]:
INDICATORS

['macd',
 'boll_ub',
 'boll_lb',
 'rsi_30',
 'cci_30',
 'dx_30',
 'close_30_sma',
 'close_60_sma']

In [101]:
stock_dimension = len(train.tic.unique())
state_space = 1 + 2*stock_dimension + len(INDICATORS)*stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")

Stock Dimension: 29, State Space: 291


In [102]:
buy_cost_list = sell_cost_list = [0.001] * stock_dimension
num_stock_shares = [0] * stock_dimension

env_kwargs = {
    "hmax": 100,
    "initial_amount": 1000000,
    "num_stock_shares": num_stock_shares,
    "buy_cost_pct": buy_cost_list,
    "sell_cost_pct": sell_cost_list,
    "state_space": state_space,
    "stock_dim": stock_dimension,
    "tech_indicator_list": INDICATORS,
    "action_space": stock_dimension,
    "reward_scaling": 1e-4
}


e_train_gym = StockTradingEnv(df = train, **env_kwargs)

## Environment for Training



In [103]:
env_train, _ = e_train_gym.get_sb_env()
print(type(env_train))

<class 'stable_baselines3.common.vec_env.dummy_vec_env.DummyVecEnv'>


<a id='5'></a>
# Part 6: Train DRL Agents
* The DRL algorithms are from **Stable Baselines 3**. Users are also encouraged to try **ElegantRL** and **Ray RLlib**.
* FinRL includes fine-tuned standard DRL algorithms, such as DQN, DDPG, Multi-Agent DDPG, PPO, SAC, A2C and TD3. We also allow users to
design their own DRL algorithms by adapting these DRL algorithms.

In [22]:
agent = DRLAgent(env = env_train)

if_using_a2c = True
if_using_ddpg = True
if_using_ppo = True
if_using_td3 = True
if_using_sac = True


### Agent Training: 5 algorithms (A2C, DDPG, PPO, TD3, SAC)


### Agent 1: A2C


In [23]:
agent = DRLAgent(env = env_train)
model_a2c = agent.get_model("a2c")

if if_using_a2c:
  # set up logger
  tmp_path = RESULTS_DIR + '/a2c'
  new_logger_a2c = configure(tmp_path, ["stdout", "csv", "tensorboard"])
  # Set new logger
  model_a2c.set_logger(new_logger_a2c)


{'n_steps': 5, 'ent_coef': 0.01, 'learning_rate': 0.0007}
Using cpu device
Logging to results/a2c


In [24]:
trained_a2c = agent.train_model(model=model_a2c, 
                             tb_log_name='a2c',
                             total_timesteps=50000) if if_using_a2c else None

----------------------------------------
| time/                 |              |
|    fps                | 86           |
|    iterations         | 100          |
|    time_elapsed       | 5            |
|    total_timesteps    | 500          |
| train/                |              |
|    entropy_loss       | -41.4        |
|    explained_variance | -0.753       |
|    learning_rate      | 0.0007       |
|    n_updates          | 99           |
|    policy_loss        | -34.1        |
|    reward             | -0.101721555 |
|    std                | 1.01         |
|    value_loss         | 3.66         |
----------------------------------------
--------------------------------------
| time/                 |            |
|    fps                | 74         |
|    iterations         | 200        |
|    time_elapsed       | 13         |
|    total_timesteps    | 1000       |
| train/                |            |
|    entropy_loss       | -41.5      |
|    explained_variance | -0.023

### Agent 2: DDPG

In [25]:
agent = DRLAgent(env = env_train)
model_ddpg = agent.get_model("ddpg")

if if_using_ddpg:
  # set up logger
  tmp_path = RESULTS_DIR + '/ddpg'
  new_logger_ddpg = configure(tmp_path, ["stdout", "csv", "tensorboard"])
  # Set new logger
  model_ddpg.set_logger(new_logger_ddpg)

{'batch_size': 128, 'buffer_size': 50000, 'learning_rate': 0.001}
Using cpu device
Logging to results/ddpg


In [26]:
trained_ddpg = agent.train_model(model=model_ddpg, 
                             tb_log_name='ddpg',
                             total_timesteps=50000) if if_using_ddpg else None

day: 2956, episode: 20
begin_total_asset: 1000000.00
end_total_asset: 2896568.03
total_reward: 1896568.03
total_cost: 1065.27
total_trades: 47321
Sharpe: 0.607
-----------------------------------
| time/              |            |
|    episodes        | 4          |
|    fps             | 24         |
|    time_elapsed    | 482        |
|    total_timesteps | 11828      |
| train/             |            |
|    actor_loss      | 3.26       |
|    critic_loss     | 41.2       |
|    learning_rate   | 0.001      |
|    n_updates       | 8871       |
|    reward          | -4.6516004 |
-----------------------------------
-----------------------------------
| time/              |            |
|    episodes        | 8          |
|    fps             | 21         |
|    time_elapsed    | 1088       |
|    total_timesteps | 23656      |
| train/             |            |
|    actor_loss      | -2.37      |
|    critic_loss     | 2.02       |
|    learning_rate   | 0.001      |
|    n_updat

### Agent 3: PPO

In [27]:
agent = DRLAgent(env = env_train)
PPO_PARAMS = {
    "n_steps": 2048,
    "ent_coef": 0.01,
    "learning_rate": 0.00025,
    "batch_size": 128,
}
model_ppo = agent.get_model("ppo",model_kwargs = PPO_PARAMS)

if if_using_ppo:
  # set up logger
  tmp_path = RESULTS_DIR + '/ppo'
  new_logger_ppo = configure(tmp_path, ["stdout", "csv", "tensorboard"])
  # Set new logger
  model_ppo.set_logger(new_logger_ppo)

{'n_steps': 2048, 'ent_coef': 0.01, 'learning_rate': 0.00025, 'batch_size': 128}
Using cpu device
Logging to results/ppo


In [28]:
trained_ppo = agent.train_model(model=model_ppo, 
                             tb_log_name='ppo',
                             total_timesteps=50000) if if_using_ppo else None

---------------------------------
| time/              |          |
|    fps             | 67       |
|    iterations      | 1        |
|    time_elapsed    | 30       |
|    total_timesteps | 2048     |
| train/             |          |
|    reward          | 2.736287 |
---------------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 69          |
|    iterations           | 2           |
|    time_elapsed         | 59          |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.012204561 |
|    clip_fraction        | 0.187       |
|    clip_range           | 0.2         |
|    entropy_loss         | -41.2       |
|    explained_variance   | -0.0229     |
|    learning_rate        | 0.00025     |
|    loss                 | 9.56        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0219     |
|    reward           

### Agent 4: TD3

In [29]:
agent = DRLAgent(env = env_train)
TD3_PARAMS = {"batch_size": 100, 
              "buffer_size": 1000000, 
              "learning_rate": 0.001}

model_td3 = agent.get_model("td3",model_kwargs = TD3_PARAMS)

if if_using_td3:
  # set up logger
  tmp_path = RESULTS_DIR + '/td3'
  new_logger_td3 = configure(tmp_path, ["stdout", "csv", "tensorboard"])
  # Set new logger
  model_td3.set_logger(new_logger_td3)

{'batch_size': 100, 'buffer_size': 1000000, 'learning_rate': 0.001}
Using cpu device
Logging to results/td3


In [ ]:
trained_td3 = agent.train_model(model=model_td3, 
                             tb_log_name='td3',
                             total_timesteps=50000) if if_using_td3 else None

-----------------------------------
| time/              |            |
|    episodes        | 4          |
|    fps             | 25         |
|    time_elapsed    | 464        |
|    total_timesteps | 11828      |
| train/             |            |
|    actor_loss      | 85.6       |
|    critic_loss     | 2.26e+03   |
|    learning_rate   | 0.001      |
|    n_updates       | 8871       |
|    reward          | -5.8078027 |
-----------------------------------
day: 2956, episode: 60
begin_total_asset: 1000000.00
end_total_asset: 3391778.54
total_reward: 2391778.54
total_cost: 999.00
total_trades: 50252
Sharpe: 0.630
-----------------------------------
| time/              |            |
|    episodes        | 8          |
|    fps             | 22         |
|    time_elapsed    | 1070       |
|    total_timesteps | 23656      |
| train/             |            |
|    actor_loss      | 59.1       |
|    critic_loss     | 438        |
|    learning_rate   | 0.001      |
|    n_update

### Agent 5: SAC

In [33]:
agent = DRLAgent(env = env_train)
SAC_PARAMS = {
    "batch_size": 128,
    "buffer_size": 100000,
    "learning_rate": 0.0001,
    "learning_starts": 100,
    "ent_coef": "auto_0.1",
}

model_sac = agent.get_model("sac",model_kwargs = SAC_PARAMS)

if if_using_sac:
  # set up logger
  tmp_path = RESULTS_DIR + '/sac'
  new_logger_sac = configure(tmp_path, ["stdout", "csv", "tensorboard"])
  # Set new logger
  model_sac.set_logger(new_logger_sac)

{'batch_size': 128, 'buffer_size': 100000, 'learning_rate': 0.0001, 'learning_starts': 100, 'ent_coef': 'auto_0.1'}
Using cpu device
Logging to results/sac


In [34]:
trained_sac = agent.train_model(model=model_sac, 
                             tb_log_name='sac',
                             total_timesteps=50000) if if_using_sac else None

day: 2956, episode: 80
begin_total_asset: 1000000.00
end_total_asset: 2788059.56
total_reward: 1788059.56
total_cost: 247103.77
total_trades: 76956
Sharpe: 0.561
----------------------------------
| time/              |           |
|    episodes        | 4         |
|    fps             | 19        |
|    time_elapsed    | 622       |
|    total_timesteps | 11828     |
| train/             |           |
|    actor_loss      | 258       |
|    critic_loss     | 22.5      |
|    ent_coef        | 0.0849    |
|    ent_coef_loss   | -113      |
|    learning_rate   | 0.0001    |
|    n_updates       | 11727     |
|    reward          | -9.282894 |
----------------------------------
-----------------------------------
| time/              |            |
|    episodes        | 8          |
|    fps             | 19         |
|    time_elapsed    | 1235       |
|    total_timesteps | 23656      |
| train/             |            |
|    actor_loss      | 107        |
|    critic_loss     | 22

## In-sample Performance

Assume that the initial capital is $1,000,000.

### Set turbulence threshold
Set the turbulence threshold to be greater than the maximum of insample turbulence data. If current turbulence index is greater than the threshold, then we assume that the current market is volatile

In [78]:
data_risk_indicator = processed_full[(processed_full.date<TRAIN_END_DATE) & (processed_full.date>=TRAIN_START_DATE)]
insample_risk_indicator = data_risk_indicator.drop_duplicates(subset=['date'])

In [79]:
insample_risk_indicator.vix.describe()

count    2957.000000
mean       18.105293
std         7.272476
min         9.140000
25%        13.370000
50%        16.209999
75%        20.629999
max        82.690002
Name: vix, dtype: float64

In [80]:
insample_risk_indicator.vix.quantile(0.996)

57.212001831054636

In [81]:
insample_risk_indicator.turbulence.describe()

count    2957.000000
mean       34.139585
std        43.879110
min         0.000000
25%        14.613506
50%        23.644663
75%        38.292580
max       652.504902
Name: turbulence, dtype: float64

In [82]:
insample_risk_indicator.turbulence.quantile(0.996)

291.72619195879605

### Trading (Out-of-sample Performance)

We update periodically in order to take full advantage of the data, e.g., retrain quarterly, monthly or weekly. We also tune the parameters along the way, in this notebook we use the in-sample data from 2009-01 to 2020-07 to tune the parameters once, so there is some alpha decay here as the length of trade date extends. 

Numerous hyperparameters – e.g. the learning rate, the total number of samples to train on – influence the learning process and are usually determined by testing some variations.

In [105]:
e_trade_gym = StockTradingEnv(df = trade, turbulence_threshold = 70,risk_indicator_col='vix', **env_kwargs)
# env_trade, obs_trade = e_trade_gym.get_sb_env()

In [106]:
trade.head()

date   tic        open        high         low       close   
0  2021-10-01  AAPL  141.899994  142.919998  139.110001  141.404266  \
0  2021-10-01  AMGN  213.589996  214.610001  210.800003  203.845871   
0  2021-10-01   AXP  168.500000  175.119995  168.479996  170.065353   
0  2021-10-01    BA  222.850006  226.720001  220.600006  226.000000   
0  2021-10-01   CAT  192.899994  195.869995  191.240005  187.928040   

       volume  day      macd     boll_ub     boll_lb     rsi_30      cci_30   
0  94639600.0  4.0 -1.703488  155.382846  137.132193  46.927735 -142.190202  \
0   2629400.0  4.0 -3.097330  212.767980  199.379595  40.408533  -96.757039   
0   3956000.0  4.0  2.273329  174.218856  149.232889  56.265093  117.538402   
0   9113600.0  4.0  0.730320  226.909442  205.727561  51.614047  116.649440   
0   3695500.0  4.0 -3.640324  205.735919  181.432783  41.999435 -112.087765   

       dx_30  close_30_sma  close_60_sma   vix  turbulence  
0  41.749873    147.171798    146.269415  21.1  120.122978  
0  36.189244    208.480832    217.103342  21.1  120.122978  
0  15.667511    161.215661    163.458888  21.1  120.122978  
0   2.027170    217.175334    221.968500  21.1  120.122978  
0  36.203176    196.993869    200.522109  21.1  120.122978

In [107]:
trained_moedl = trained_a2c
df_account_value_a2c, df_actions_a2c = DRLAgent.DRL_prediction(
    model=trained_moedl, 
    environment = e_trade_gym)

hit end!


In [108]:
trained_moedl = trained_ddpg
df_account_value_ddpg, df_actions_ddpg = DRLAgent.DRL_prediction(
    model=trained_moedl, 
    environment = e_trade_gym)

hit end!


In [109]:
trained_moedl = trained_ppo
df_account_value_ppo, df_actions_ppo = DRLAgent.DRL_prediction(
    model=trained_moedl, 
    environment = e_trade_gym)

hit end!


In [110]:
trained_moedl = trained_td3
df_account_value_td3, df_actions_td3 = DRLAgent.DRL_prediction(
    model=trained_moedl, 
    environment = e_trade_gym)

hit end!


In [111]:
trained_moedl = trained_sac
df_account_value_sac, df_actions_sac = DRLAgent.DRL_prediction(
    model=trained_moedl, 
    environment = e_trade_gym)

hit end!


In [112]:
df_account_value_a2c.shape

(353, 2)

<a id='7'></a>
# Part 6.5: Mean Variance Optimization

Mean Variance optimization is a very classic strategy in portfolio management. Here, we go through the whole process to do the mean variance optimization and add it as a baseline to compare.

First, process dataframe to the form for MVO weight calculation.

In [134]:
def process_df_for_mvo(df):
  df = df.sort_values(['date','tic'],ignore_index=True)[['date','tic','close']]
  fst = df
  fst = fst.iloc[0:stock_dimension, :]
  tic = fst['tic'].tolist()

  mvo = pd.DataFrame()

  for k in range(len(tic)):
    mvo[tic[k]] = 0

  for i in range(df.shape[0]//stock_dimension):
    n = df
    n = n.iloc[i * stock_dimension:(i+1) * stock_dimension, :]
    date = n['date'][i*stock_dimension]
    mvo.loc[date] = n['close'].tolist()
  
  return mvo

### Helper functions for mean returns and variance-covariance matrix

In [135]:
# Codes in this section partially refer to Dr G A Vijayalakshmi Pai

# https://www.kaggle.com/code/vijipai/lesson-5-mean-variance-optimization-of-portfolios/notebook

def StockReturnsComputing(StockPrice, Rows, Columns): 
  import numpy as np 
  StockReturn = np.zeros([Rows-1, Columns]) 
  for j in range(Columns):        # j: Assets 
    for i in range(Rows-1):     # i: Daily Prices 
      StockReturn[i,j]=((StockPrice[i+1, j]-StockPrice[i,j])/StockPrice[i,j])* 100 
      
  return StockReturn

### Calculate the weights for mean-variance

In [138]:
train_mvo = data_split(processed_full, TRAIN_START_DATE,TRAIN_END_DATE).reset_index()
trade_mvo = data_split(processed_full, TRADE_START_DATE,TRADE_END_DATE).reset_index()

In [139]:
StockData = process_df_for_mvo(train_mvo)
TradeData = process_df_for_mvo(trade_mvo)

TradeData.to_numpy()

array([[141.404, 203.846, 170.065, ...,  49.275,  43.724, 133.839],
       [137.925, 201.483, 168.814, ...,  49.456,  43.621, 132.55 ],
       [139.878, 201.883, 170.867, ...,  49.465,  43.995, 133.419],
       ...,
       [149.4  , 237.62 , 174.494, ...,  37.903,  36.21 , 141.51 ],
       [146.71 , 233.66 , 173.607, ...,  38.109,  35.8  , 141.888],
       [147.92 , 234.45 , 172.66 , ...,  38.247,  35.39 , 140.863]])

In [65]:
#compute asset returns
arStockPrices = np.asarray(StockData)
[Rows, Cols]=arStockPrices.shape
arReturns = StockReturnsComputing(arStockPrices, Rows, Cols)

#compute mean returns and variance covariance matrix of returns
meanReturns = np.mean(arReturns, axis = 0)
covReturns = np.cov(arReturns, rowvar=False)
 
#set precision for printing results
np.set_printoptions(precision=3, suppress = True)

#display mean returns and variance-covariance matrix of returns
print('Mean returns of assets in k-portfolio 1\n', meanReturns)
print('Variance-Covariance matrix of returns\n', covReturns)

Mean returns of assets in k-portfolio 1
 [0.12  0.065 0.07  0.08  0.068 0.116 0.051 0.039 0.073 0.049 0.102 0.078
 0.024 0.06  0.048 0.072 0.039 0.064 0.045 0.048 0.096 0.092 0.046 0.058
 0.104 0.094 0.043 0.034 0.048]
Variance-Covariance matrix of returns
 [[3.14  1.017 1.294 1.529 1.374 1.696 1.379 1.13  1.092 1.377 1.165 1.224
  1.035 1.526 0.693 1.272 0.667 0.819 1.032 0.737 1.579 1.145 0.672 0.863
  1.146 1.34  0.526 0.909 0.635]
 [1.017 2.406 1.076 1.016 1.067 1.218 1.027 0.967 0.938 1.128 0.984 1.032
  0.851 1.155 0.892 1.143 0.643 0.668 0.909 1.074 1.092 0.879 0.727 0.855
  1.147 1.058 0.629 1.027 0.648]
 [1.294 1.076 3.307 2.486 1.912 1.575 1.435 1.926 1.721 2.24  1.382 1.859
  1.342 1.534 0.866 2.37  0.994 1.082 1.384 0.959 1.362 1.424 0.73  1.509
  1.416 1.802 0.75  1.223 0.586]
 [1.529 1.016 2.486 4.984 2.129 1.698 1.518 2.153 1.845 2.214 1.537 2.132
  1.536 1.706 0.911 2.308 1.123 1.201 1.47  0.947 1.444 1.583 0.752 1.613
  1.483 1.722 0.738 1.374 0.609]
 [1.374 1.067 1.91

### Use PyPortfolioOpt

In [66]:
from pypfopt.efficient_frontier import EfficientFrontier

ef_mean = EfficientFrontier(meanReturns, covReturns, weight_bounds=(0, 0.5))
raw_weights_mean = ef_mean.max_sharpe()
cleaned_weights_mean = ef_mean.clean_weights()
mvo_weights = np.array([1000000 * cleaned_weights_mean[i] for i in range(29)])
mvo_weights

array([288130.,      0.,      0.,      0.,      0.,  69330.,      0.,
            0.,      0.,      0., 316160.,      0.,      0.,      0.,
            0.,      0.,      0.,      0.,      0.,      0.,      0.,
        66290.,      0.,      0., 237410.,  22690.,      0.,      0.,
            0.])

In [67]:
LastPrice = np.array([1/p for p in StockData.tail(1).to_numpy()[0]])
Initial_Portfolio = np.multiply(mvo_weights, LastPrice)
Initial_Portfolio

array([2054.193,    0.   ,    0.   ,    0.   ,    0.   ,  255.623,
          0.   ,    0.   ,    0.   ,    0.   ,  998.634,    0.   ,
          0.   ,    0.   ,    0.   ,    0.   ,    0.   ,    0.   ,
          0.   ,    0.   ,    0.   ,  463.5  ,    0.   ,    0.   ,
        619.578,  103.016,    0.   ,    0.   ,    0.   ])

In [68]:
Portfolio_Assets = TradeData @ Initial_Portfolio
MVO_result = pd.DataFrame(Portfolio_Assets, columns=["Mean Var"])
# MVO_result

<a id='6'></a>
# Part 7: Backtesting Results
Backtesting plays a key role in evaluating the performance of a trading strategy. Automated backtesting tool is preferred because it reduces the human error. We usually use the Quantopian pyfolio package to backtest our trading strategies. It is easy to use and consists of various individual plots that provide a comprehensive image of the performance of a trading strategy.

In [145]:
df_result_a2c = df_account_value_a2c.set_index(df_account_value_a2c.columns[0])
df_result_a2c.rename(columns = {'account_value':'a2c'}, inplace = True)
df_result_ddpg = df_account_value_ddpg.set_index(df_account_value_ddpg.columns[0])
df_result_ddpg.rename(columns = {'account_value':'ddpg'}, inplace = True)
df_result_td3 = df_account_value_td3.set_index(df_account_value_td3.columns[0])
df_result_td3.rename(columns = {'account_value':'td3'}, inplace = True)
df_result_ppo = df_account_value_ppo.set_index(df_account_value_ppo.columns[0])
df_result_ppo.rename(columns = {'account_value':'ppo'}, inplace = True)
df_result_sac = df_account_value_sac.set_index(df_account_value_sac.columns[0])
df_result_sac.rename(columns = {'account_value':'sac'}, inplace = True)
df_account_value_a2c.to_csv("df_account_value_a2c.csv")
#baseline stats
print("==============Get Baseline Stats===========")
df_dji_ = get_baseline(
        ticker="^DJI", 
        start = TRADE_START_DATE,
        end = TRADE_END_DATE)
stats = backtest_stats(df_dji_, value_col_name = 'close')
df_dji = pd.DataFrame()
df_dji['date'] = df_account_value_a2c['date']
df_dji['account_value'] = df_dji_['close'] / df_dji_['close'][0] * env_kwargs["initial_amount"]
df_dji.to_csv("df_dji.csv")
df_dji = df_dji.set_index(df_dji.columns[0])
df_dji.to_csv("df_dji+.csv")

result = pd.DataFrame()
result = pd.merge(result, df_result_a2c, how='outer', left_index=True, right_index=True)
result = pd.merge(result, df_result_ddpg, how='outer', left_index=True, right_index=True)
result = pd.merge(result, df_result_td3, how='outer', left_index=True, right_index=True)
result = pd.merge(result, df_result_ppo, how='outer', left_index=True, right_index=True)
result = pd.merge(result, df_result_sac, how='outer', left_index=True, right_index=True)
result = pd.merge(result, MVO_result, how='outer', left_index=True, right_index=True)
print(result.head())
result = pd.merge(result, df_dji, how='outer', left_index=True, right_index=True)
# result.columns = ['a2c', 'ddpg', 'td3', 'ppo', 'sac', 'mean var', 'dji']

# print("result: ", result)
result.to_csv("result.csv")

==============Get Baseline Stats===========
[*********************100%***********************]  1 of 1 completed
Shape of DataFrame:  (354, 8)
Annual return         -0.034876
Cumulative returns    -0.048644
Annual volatility      0.181612
Sharpe ratio          -0.105351
Calmar ratio          -0.158953
Stability              0.280983
Max drawdown          -0.219408
Omega ratio            0.982546
Sortino ratio         -0.146974
Skew                        NaN
Kurtosis                    NaN
Tail ratio             0.970602
Daily value at risk   -0.022957
dtype: float64
                     a2c          ddpg           td3           ppo   
date                                                                 
2021-10-01  1.000000e+06  1.000000e+06  1.000000e+06  1.000000e+06  \
2021-10-04  9.971647e+05  9.977126e+05  9.961645e+05  9.995875e+05   
2021-10-05  1.000508e+06  1.003342e+06  1.002158e+06  1.000310e+06   
2021-10-06  1.002597e+06  1.004220e+06  1.006038e+06  1.000651e+06   
2021-1

In [141]:
df_result_ddpg

ddpg
date                    
2021-10-01  1.000000e+06
2021-10-04  9.977126e+05
2021-10-05  1.003342e+06
2021-10-06  1.004220e+06
2021-10-07  1.010823e+06
...                  ...
2023-02-21  9.815355e+05
2023-02-22  9.794530e+05
2023-02-23  9.820209e+05
2023-02-24  9.739605e+05
2023-02-27  9.752743e+05

[353 rows x 1 columns]

In [144]:
%matplotlib inline
plt.rcParams["figure.figsize"] = (15,5)
plt.figure();
result.plot();

<Figure size 1500x500 with 0 Axes>